In [ ]:
pip install djangorestframework

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'rest_framework.authtoken',
    'cats.apps.CatsConfig',
]

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated', 
    ],

    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.TokenAuthentication',
    ]
}

In [ ]:
python manage.py migrate

In [ ]:
from rest_framework.authtoken import views


urlpatterns = [

    ...

    path('api-token-auth/', views.obtain_auth_token),
]

![alt text](https://pictures.s3.yandex.net/resources/S09_34_1700216480.png)


***
## Практика: подключите и настройте JWT-аутентификацию

Для работы с JWT в Django установите и подключите две библиотеки [Djoser](https://djoser.readthedocs.io/en/latest/getting_started.html) и [Simple JWT](https://django-rest-framework-simplejwt.readthedocs.io/en/latest/):


In [ ]:

pip install djoser djangorestframework_simplejwt 


Обновите файл *settings.py*:


In [ ]:

INSTALLED_APPS = (
    'django.contrib.auth',
    ...
    'rest_framework',
    'djoser',
) 


Обратите внимание, приложение Djoser должно быть зарегистрировано после `django.contrib.auth` и `rest_framework`.

Добавьте новые настройки в *settings.py*, они сходны с настройками Authtoken:

* permission;

* способ аутентификации по умолчанию;

* минимально необходимые настройки модуля Simple JWT.


In [ ]:

from datetime import timedelta

...

REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated', 
    ],

    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    ],
}

SIMPLE_JWT = {
    # Устанавливаем срок жизни токена
    'ACCESS_TOKEN_LIFETIME': timedelta(days=1),
    'AUTH_HEADER_TYPES': ('Bearer',),
} 


Проверьте, нет ли в проекте неприменённых миграций: `python manage.py migrate`.

Измените файл роутинга urls.py:


In [ ]:

urlpatterns = [
    ...
    # Djoser создаст набор необходимых эндпоинтов.
    # базовые, для управления пользователями в Django:
    path('auth/', include('djoser.urls')),
    # JWT-эндпоинты, для управления JWT-токенами:
    path('auth/', include('djoser.urls.jwt')),
] 

In [ ]:
from rest_framework.permissions import IsAuthenticated

@permission_classes([IsAuthenticated])  # доступ только авторизованным пользователям

![alt text](https://pictures.s3.yandex.net/resources/S09_49_1700216544.png)


Список всех эндпоинтов, которые создаёт **djoser**, есть [в документации](https://djoser.readthedocs.io/en/latest/getting_started.html#available-endpoints). 

Если нужно изменить набор полей сериализатора из **djoser**, то

* из `djoser.serializers` импортируется класс сериализатора, который нужно переопределить (например, `UserSerializer` или `UserCreateSerializer`; полный список сериализаторов **djoser** доступен [в документации](https://djoser.readthedocs.io/en/latest/settings.html#serializers));

* описывается новый класс сериализатора (он наследуется от импортированного);

* в новом сериализаторе переопределяется набор полей, используемых по умолчанию.


In [ ]:

from djoser.serializers import UserSerializer
...

class CustomUserSerializer(UserSerializer):
    class Meta:
        model = User
        fields = ('email', 'id', 'username', 'first_name', 'last_name') 


Аналогично можно поступить и с вьюсетом, который использовал сериализатор.


In [ ]:

from djoser.views import UserViewSet
...
from .serializers import CustomUserSerializer
...

class CustomUserViewSet(UserViewSet):
    ...
 

***
## Применение JWT на практике

Теперь пользователя можно создать через API. 

Придумайте новую пару «логин-пароль» и отправьте POST-запрос на http://127.0.0.1:8000/auth/users/, передав их в полях `username` и `password`.

Теперь можно получить токен: отправьте POST-запрос на эндпоинт */auth/jwt/create/*, передав действующий логин и пароль в полях `username` и `password`.

API вернёт JWT-токен:


Токен вернётся в поле `access`, а данные из поля `refresh` пригодятся для обновления токена. 

При каждом запросе к API нужно в заголовке запроса, в поле **Authorization**, передавать основной токен доступа, полученный в поле `access`. Перед самим токеном должно стоять ключевое слово `Bearer` и пробел: `Bearer токен`

In [ ]:
from django.shortcuts import get_object_or_404
from rest_framework.decorators import api_view, permission_classes
from rest_framework.response import Response
from rest_framework import status
from .models import Post
from rest_framework.permissions import IsAuthenticated
from .serializers import PostSerializer


@api_view(['GET', 'POST'])
def api_posts(request):
    if request.method == 'POST':
        serializer = PostSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save(author=request.user)
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    posts = Post.objects.all()
    serializer = PostSerializer(posts, many=True)
    return Response(serializer.data, status=status.HTTP_200_OK)


@api_view(['GET', 'PUT', 'PATCH', 'DELETE'])
@permission_classes([IsAuthenticated])
def api_posts_detail(request, pk):
    post = get_object_or_404(Post, id=pk)

    if request.method == 'GET':
        serializer = PostSerializer(post)
        return Response(serializer.data, status=status.HTTP_200_OK)


    if request.user != post.author:
            return Response(
                {'detail': 'Изменять или удалять пост может только автор.'},
                status=status.HTTP_403_FORBIDDEN
            )

    if request.method == 'PUT' or request.method == 'PATCH':
        
        serializer = PostSerializer(post, data=request.data, partial=True)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        post.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)